--- 

### Importing and filtering data:

In [16]:
import pandas as pd
import numpy as np 

import_file_path = r"C:\Users\olive\OneDrive\Documents\University Work\Y3S2\JIV_RA_task\usa_wk_apr_2013.xlsx"
export_file_path = r"C:\Users\olive\OneDrive\Documents\University Work\Y3S2\JIV_RA_task\KLEMS_indeces_2013.xlsx"

KLEMS_final = (
    pd.read_excel(import_file_path, sheet_name="DATA")
    .query("code == 'AtB'")
    .melt(id_vars=["Variable", "desc", "code"], var_name ="Year", value_name="Value")
    .pivot(index="Year", columns="Variable", values="Value")
    .loc[:, ["GO", "GO_QI", "II", "II_QI", "LAB", "LAB_QI", "CAP", "CAP_QI", "H_EMP"]])
KLEMS_final.index = KLEMS_final.index.astype(str).str.replace("_","")

--- 

### Start of main task:

--- 

### Compute nominal value added and nominal value added share of output

#### Nominal value added

Formula

$$
P_{V A}(t) Q_{V A}(t) = P_Y (t) Q_Y (t) - P_{II} (t) Q_{II} (t)
$$

In [17]:
KLEMS_final.loc[:, "nominal_VA"] = (KLEMS_final["GO"] - KLEMS_final["II"])

#### Nominal value added share of output

Formula: 

$$
\nu_{V A}(t) = \frac{P_{V A}(t) Q_{V A}(t)}{P_Y (t) Q_Y (t)}
$$

In [18]:
KLEMS_final.loc[:, "VA_output_share"] = (KLEMS_final["nominal_VA"]/KLEMS_final["GO"])

---

### Value added quantity growth rates

Formula: 

$$
\Delta \ln Q_{VA}(t) = \frac{\Delta \ln Q_Y (t) - \bar{\nu}_{II} (t) \Delta \ln Q_{II} (t)}{\bar{\nu}_{V A} (t)}
$$

where:

$$
\Delta \ln X(t) = \ln X(t) - \ln X(t - 1)
$$

and:

$$
\bar{\nu}_X(t) = 0.5 \times \left( \frac{P_X(t) Q_X(t)}{P_Y (t) Q_Y (t)} + \frac{P_X(t - 1) Q_X(t - 1)}{P_Y (t - 1) Q_Y (t - 1)} \right)
$$

#### Log Differences

$$
\Delta \ln X(t) = \ln X(t) - \ln X(t - 1)
$$

where X is either real GO or real II

In [19]:
KLEMS_final.loc[:, "ln_GO_QI"] = np.log(KLEMS_final["GO_QI"])
KLEMS_final.loc[:, "delta_ln_GO_QI"] = (KLEMS_final["ln_GO_QI"] - KLEMS_final["ln_GO_QI"].shift(1))

KLEMS_final.loc[:, "ln_II_QI"] = np.log(KLEMS_final["II_QI"])
KLEMS_final.loc[:, "delta_ln_II_QI"] = (KLEMS_final["ln_II_QI"] - KLEMS_final["ln_II_QI"].shift(1))

#### Tornqvist Output Share

$$
\bar{\nu}_X(t) = 0.5 \times \left( \frac{P_X(t) Q_X(t)}{P_Y (t) Q_Y (t)} + \frac{P_X(t - 1) Q_X(t - 1)}{P_Y (t - 1) Q_Y (t - 1)} \right)
$$

where X is either nominal VA or nominal II

In [20]:
KLEMS_final.loc[:, "VA_tornqvist_output_share"] = (0.5*((KLEMS_final["nominal_VA"]/KLEMS_final["GO"])+(KLEMS_final["nominal_VA"].shift(1)/KLEMS_final["GO"].shift(1))))
KLEMS_final.loc[:, "II_tornqvist_output_share"] = (0.5*((KLEMS_final["II"]/KLEMS_final["GO"])+(KLEMS_final["II"].shift(1)/KLEMS_final["GO"].shift(1))))

#### Real VA Growth Rate

$$
\Delta \ln Q_{VA}(t) = \frac{\Delta \ln Q_Y (t) - \bar{\nu}_{II} (t) \Delta \ln Q_{II} (t)}{\bar{\nu}_{V A} (t)}
$$

In [21]:
KLEMS_final.loc[:, "delta_ln_Q_va"] = ((KLEMS_final["delta_ln_GO_QI"] - (KLEMS_final["II_tornqvist_output_share"]*KLEMS_final["delta_ln_II_QI"]))/KLEMS_final["VA_tornqvist_output_share"])

--- 

### Labour Productivity (LP) growth rates

Formula:

$$
\Delta \ln LP(t) = \Delta \ln Q_{VA}(t) - \Delta \ln Q_L(t)
$$

#### Using LAB_QI:

In [22]:
KLEMS_final.loc[:, "ln_LAB_QI"] = np.log(KLEMS_final["LAB_QI"])
KLEMS_final.loc[:, "delta_ln_LAB_QI"] = (KLEMS_final["ln_LAB_QI"] - KLEMS_final["ln_LAB_QI"].shift(1))
KLEMS_final.loc[:, "delta_ln_LP"] = (KLEMS_final["delta_ln_Q_va"] - KLEMS_final["delta_ln_LAB_QI"])

#### Using H_EMP:

In [23]:
KLEMS_final.loc[:, "ln_H_EMP"] = np.log(KLEMS_final["H_EMP"])
KLEMS_final.loc[:, "delta_ln_H_EMP"] = (KLEMS_final["ln_H_EMP"] - KLEMS_final["ln_H_EMP"].shift(1))
KLEMS_final.loc[:, "delta_ln_LP_using_H_EMP"] = (KLEMS_final["delta_ln_Q_va"] - KLEMS_final["delta_ln_H_EMP"])

--- 

### Total Factor Productivity (TFP) growth rate

TFP formula:
$$
\Delta \ln TFP(t) = \Delta \ln Q_{VA}(t) - \bar{\psi}_L(t) \Delta \ln Q_L(t) - \bar{\psi}_K(t) \Delta \ln Q_K(t)
$$

where:

$$
\bar{\psi}_X(t) = 0.5 \times \left( \frac{P_X(t) Q_X(t)}{P_{VA}(t) Q_{VA}(t)} + \frac{P_X(t-1) Q_X(t-1)}{P_{VA}(t-1) Q_{VA}(t-1)} \right)
$$

#### Tornqvist VA Share

$$
\bar{\psi}_X(t) = 0.5 \times \left( \frac{P_X(t) Q_X(t)}{P_{VA}(t) Q_{VA}(t)} + \frac{P_X(t-1) Q_X(t-1)}{P_{VA}(t-1) Q_{VA}(t-1)} \right)
$$

where X is either nominal LAB (L) or nominal CAP (K)

In [24]:
KLEMS_final.loc[:, "L_tornqvist_VA_share"] = (0.5*((KLEMS_final["LAB"]/KLEMS_final["nominal_VA"])+(KLEMS_final["LAB"].shift(1)/KLEMS_final["nominal_VA"].shift(1))))
KLEMS_final.loc[:, "K_tornqvist_VA_share"] = (0.5*((KLEMS_final["CAP"]/KLEMS_final["nominal_VA"])+(KLEMS_final["CAP"].shift(1)/KLEMS_final["nominal_VA"].shift(1))))

Real capital; logged and log difference:

In [25]:
KLEMS_final.loc[:, "ln_CAP_QI"] = np.log(KLEMS_final["CAP_QI"])
KLEMS_final.loc[:, "delta_ln_CAP_QI"] = (KLEMS_final["ln_CAP_QI"] - KLEMS_final["ln_CAP_QI"].shift(1))

#### TFP growth rate

$$
\Delta \ln TFP(t) = \Delta \ln Q_{VA}(t) - \bar{\psi}_L(t) \Delta \ln Q_L(t) - \bar{\psi}_K(t) \Delta \ln Q_K(t)
$$

In [26]:
KLEMS_final.loc[:, "delta_ln_TFP"] = (KLEMS_final["delta_ln_Q_va"] - (KLEMS_final["L_tornqvist_VA_share"]*KLEMS_final["delta_ln_LAB_QI"]) - (KLEMS_final["K_tornqvist_VA_share"]*KLEMS_final["delta_ln_CAP_QI"]))

--- 

### Turning Growth Rates Into Indeces

In [27]:
# Cumulatively sums the yearly changes in TFP and LP, starting from 1947 = 0. 
KLEMS_final.loc[:, "ln_TFP"] = KLEMS_final["delta_ln_TFP"].cumsum()
KLEMS_final.loc[:, "ln_LP"] = KLEMS_final["delta_ln_LP"].cumsum()
KLEMS_final.loc[:, "ln_LP_using_H_EMP"] = KLEMS_final["delta_ln_LP_using_H_EMP"].cumsum()

In [28]:
# Applies exponential operator to log indeces:
KLEMS_final.loc[:, "index_ln_TFP"] = np.exp(KLEMS_final["ln_TFP"])
KLEMS_final.loc[:, "index_ln_LP"] = np.exp(KLEMS_final["ln_LP"])
KLEMS_final.loc[:, "index_ln_LP_using_H_EMP"] = np.exp(KLEMS_final["ln_LP_using_H_EMP"])

In [29]:
KLEMS_final.iloc[0, KLEMS_final.columns.get_indexer(['index_ln_TFP', 'index_ln_LP', 'index_ln_LP_using_H_EMP'])] = 1

--- 

### Export to Excel

In [30]:
columns = ["index_ln_TFP", "index_ln_LP", "index_ln_LP_using_H_EMP"] 

KLEMS_final[columns].to_excel(export_file_path, sheet_name="Data", index=True) # Note: Year is index, so Year is also exported with index=True

print(f"File successfully saved to: {export_file_path}")

File successfully saved to: C:\Users\olive\OneDrive\Documents\University Work\Y3S2\JIV_RA_task\KLEMS_indeces_2013.xlsx
